<a href="https://colab.research.google.com/github/LeVanDuy3397/Yolov8/blob/main/Yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class Conv(nn.Module):
    """Standard convolution block: Conv2d -> BatchNorm -> SiLU"""
    def __init__(self, in_channels, out_channels, kernel_size=1, stride=1, padding=None):
        super().__init__()
        if padding is None:
            padding = kernel_size // 2
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.SiLU()

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))


In [ ]:
class C2f(nn.Module):
    """C2f module: Cross Stage Partial with concatenation (used in YOLOv8)"""
    def __init__(self, in_channels, out_channels, n=1, shortcut=True):
        super().__init__()
        hidden_channels = int(out_channels / 2)
        self.cv1 = Conv(in_channels, 2 * hidden_channels, 1, 1)
        self.cv2 = Conv((n + 2) * hidden_channels, out_channels, 1)
        self.m = nn.ModuleList(Conv(hidden_channels, hidden_channels, 3) for _ in range(n))
        self.shortcut = shortcut

    def forward(self, x):
        y = list(torch.chunk(self.cv1(x), 2, dim=1))
        for m in self.m:
            y.append(m(y[-1]) if self.shortcut else m(y[1]))
        return self.cv2(torch.cat(y, 1))

In [ ]:
class SPPF(nn.Module):
    """SPPF module: Spatial Pyramid Pooling - Fast"""
    def __init__(self, in_channels, out_channels, k=5):
        super().__init__()
        self.cv1 = Conv(in_channels, in_channels // 2, 1, 1)
        self.cv2 = Conv(in_channels * 2, out_channels, 1, 1)
        self.m = nn.MaxPool2d(kernel_size=k, stride=1, padding=k // 2)

    def forward(self, x):
        x = self.cv1(x)
        y1 = self.m(x)
        y2 = self.m(y1)
        y3 = self.m(y2)
        return self.cv2(torch.cat([x, y1, y2, y3], 1))

In [ ]:
# Define the simplified YOLOv8 Backbone
class YOLOv8Backbone(nn.Module):
    def __init__(self, ch=3):
        super().__init__()
        self.stem = Conv(ch, 32, 3, 2)
        self.stage1 = nn.Sequential(
            Conv(32, 64, 3, 2),
            C2f(64, 64, n=1)
        )
        self.stage2 = nn.Sequential(
            Conv(64, 128, 3, 2),
            C2f(128, 128, n=2)
        )
        self.stage3 = nn.Sequential(
            Conv(128, 256, 3, 2),
            C2f(256, 256, n=3)
        )
        self.stage4 = nn.Sequential(
            Conv(256, 512, 3, 2),
            C2f(512, 512, n=1),
            SPPF(512, 512)
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x_small = self.stage3(x)
        x_large = self.stage4(x_small)
        return x_small, x_large

In [ ]:
class YOLOv8Neck(nn.Module):
    def __init__(self):
        super().__init__()
        self.reduce_conv1 = Conv(512, 256, 1, 1)
        self.reduce_conv2 = Conv(256, 128, 1, 1)

        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')

        self.c2f1 = C2f(256 + 256, 256, n=1)
        self.c2f2 = C2f(128 + 128, 128, n=1)

        self.down_conv1 = Conv(128, 128, 3, 2)
        self.down_conv2 = Conv(256, 256, 3, 2)

        self.c2f3 = C2f(128 + 256, 256, n=1)
        self.c2f4 = C2f(256 + 512, 512, n=1)

    def forward(self, x_small, x_large):
        x = self.reduce_conv1(x_large)
        x = self.upsample(x)
        x = torch.cat([x, x_small], dim=1)
        x = self.c2f1(x)

        x_small_up = self.reduce_conv2(x)
        x_small_up = self.upsample(x_small_up)
        x_small_cat = torch.cat([x_small_up, x_small_up], dim=1)
        x_small_out = self.c2f2(x_small_cat)

        x_down1 = self.down_conv1(x_small_out)
        x_down1 = torch.cat([x_down1, x], dim=1)
        x_down1 = self.c2f3(x_down1)

        x_down2 = self.down_conv2(x_down1)
        x_down2 = torch.cat([x_down2, x_large], dim=1)
        x_down2 = self.c2f4(x_down2)

        return x_small_out, x_down1, x_down2

In [ ]:
class DetectHead(nn.Module):
    def __init__(self, num_classes=80, ch=[128, 256, 512]):
        super().__init__()
        self.detect_layers = nn.ModuleList()
        for c in ch:
            self.detect_layers.append(
                nn.Conv2d(c, num_classes + 5, 1)
            )

    def forward(self, features):
        outputs = []
        for x, conv in zip(features, self.detect_layers):
            out = conv(x)
            bs, _, h, w = out.shape
            out = out.view(bs, -1, h * w).permute(0, 2, 1)
            outputs.append(out)
        return torch.cat(outputs, 1)

In [ ]:
class YOLOv8(nn.Module):
    def __init__(self, num_classes=80):
        super().__init__()
        self.backbone = YOLOv8Backbone()
        self.neck = YOLOv8Neck()
        self.head = DetectHead(num_classes=num_classes)

    def forward(self, x):
        x_small, x_large = self.backbone(x)
        features = self.neck(x_small, x_large)
        out = self.head(features)
        return out

In [ ]:
# Instantiate and test
model = YOLOv8(num_classes=80)
# hàm randn sẽ lấy 1 tensor đầu vào với giá trị ngẫu nhiên, còn bên trong là hình dạng của tensor
# 1 là duy nhất 1 tensor, 3 là số kênh, 640x640 là chiều cao và rộng
dummy_input = torch.randn(1, 3, 640, 640)
# còn tensor đầu vào sẽ lưu trữ dữ liệu hình ảnh dưới dạng các pixel, cụ thể đầu tiên là xác định pixel của
# hình ảnh nào (ở đây là chỉ có 1 hình ảnh), sau đó xác định kênh màu của pixel đó,
# cuối cùng xác định vị trí pixel trong khoảng 640x640
# còn bản chất tensor đầu vào này chưa phải feature maps, nó mới là dữ liệu hình ảnh thô đầu vào
# còn bản chất feature maps sẽ được tạo ra sau khi dữ liệu hình ảnh được xử lý qua các lớp conv của model
# mỗi lớp conv sẽ trích xuất đặc trưng khác nhau của ảnh tạo ra tập hợp các feature maps

# tổng quan mô hình: ảnh đầu vào là bức tranh phong cảnh -> feature maps là các bản phác thảo của bức tranh
# -> còn model là 1 họa sĩ, sẽ dùng các bản phác thảo này để hiểu và phân tích bức tranh
output = model(dummy_input)
print(output.shape)
# kết quả là torch.Size([1, 8400, 85])
# với 1 là chuyển 1 hình ảnh duy nhất
# với 8400 là số lượng hộp giới hạn dự đoán cho 1 hình ảnh đầu vào
# và với mỗi hộp giới hạn có 85 giá trị liên quan đến vị trí, độ tin cậy của đối tượng và xác suất lớp

torch.Size([1, 8400, 85])


In [ ]:
#1) nạp parameter từ model yolov8 có sẵn đã được huấn luyện trước

#2) sau đó chạy inference, inference ở đây hiểu là suy luận tức là quá trình sử dụng
# 1 model đã được train trước sau đó dự đoán với dữ liệu mới, vậy bây giờ sử dụng dữ liệu
# thu thập được từ laptop cá nhân sau đó đưa ra dự đoán với model của mình

In [ ]:
!pip uninstall torch -y

Found existing installation: torch 2.6.0+cu118
Uninstalling torch-2.6.0+cu118:
  Successfully uninstalled torch-2.6.0+cu118


In [ ]:
!pip uninstall torchvision -y

Found existing installation: torchvision 0.21.0+cu118
Uninstalling torchvision-0.21.0+cu118:
  Successfully uninstalled torchvision-0.21.0+cu118


In [ ]:
!pip install torchvision --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.21.0%2Bcu118-cp311-cp311-linux_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cu118/torch-2.6.0%2Bcu118-cp311-cp311-linux_x86_64.whl.metadata (27 kB)
Using cached https://download.pytorch.org/whl/cu118/torchvision-0.21.0%2Bcu118-cp311-cp311-linux_x86_64.whl (6.5 MB)
Using cached https://download.pytorch.org/whl/cu118/torch-2.6.0%2Bcu118-cp311-cp311-linux_x86_64.whl (848.7 MB)


In [ ]:
!pip install numpy

In [ ]:
from PIL import Image
from torchvision import transforms
import numpy as np  # Import numpy

image = Image.open("toi.jpg")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((640, 640)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [21]:
tensor = transform(image)
print(tensor)

tensor([[[ 1.4677,  1.5108,  1.5151,  ..., -0.8105, -0.8248, -0.8361],
         [ 1.4410,  1.4810,  1.4895,  ..., -0.8228, -0.8154, -0.8329],
         [ 1.4223,  1.4666,  1.4955,  ..., -0.7876, -0.7686, -0.7917],
         ...,
         [ 2.0728,  2.0968,  2.1056,  ..., -0.9835, -0.8785, -0.8672],
         [ 2.0709,  2.0942,  2.1033,  ..., -0.9352, -0.8956, -0.9088],
         [ 2.0371,  2.0854,  2.1106,  ..., -0.8807, -0.8627, -0.8742]],

        [[ 1.6274,  1.6543,  1.6303,  ..., -0.3359, -0.3483, -0.3727],
         [ 1.6001,  1.6238,  1.6052,  ..., -0.3485, -0.3452, -0.3707],
         [ 1.5810,  1.6091,  1.6146,  ..., -0.3125, -0.3171, -0.3323],
         ...,
         [ 2.3986,  2.3847,  2.3871,  ..., -0.4799, -0.4119, -0.4069],
         [ 2.3629,  2.3771,  2.3815,  ..., -0.4601, -0.4344, -0.4495],
         [ 2.3170,  2.3664,  2.3879,  ..., -0.4142, -0.4023, -0.4141]],

        [[ 1.8847,  1.9542,  1.9672,  ..., -0.2364, -0.2770, -0.3057],
         [ 1.8576,  1.9238,  1.9417,  ..., -0

In [27]:
num_keys = len(model.state_dict().keys())
print(f"Số lượng keys: {num_keys}")

Số lượng keys: 234


In [29]:
print(model)

YOLOv8(
  (backbone): YOLOv8Backbone(
    (stem): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (stage1): Sequential(
      (0): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (1): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
      